In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
dt=pd.read_csv("fake_and_real_news.csv")
dt.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [ ]:
# dt.drop(['id','title'],axis=1,inplace=True)
# dt.head()

In [ ]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9900 entries, 0 to 9899
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9900 non-null   object
 1   label   9900 non-null   object
dtypes: object(2)
memory usage: 154.8+ KB


In [ ]:
print("nulll values \n",dt.isnull().sum())
print("duplication \n",dt.duplicated().sum())

nulll values 
 Text     0
label    0
dtype: int64
duplication 
 35


In [ ]:
# mode1=dt['text'].mode()[0]
# dt.fillna(mode1,inplace=True)
dt.drop_duplicates(inplace=True)
# print("null values now",dt.isnull().sum())
print("duplication \n",dt.duplicated().sum())


duplication 
 0


In [ ]:
# dt['real']=pd.to_numeric(dt['real'], errors='coerce').astype(int)

In [ ]:
stemmer=PorterStemmer()
lemmatizer=WordNetLemmatizer()
corpus=[]
for index,val in dt.iterrows():
  review=re.sub("[^a-zA-z]",'',dt['Text'][index])
  review=review.lower()
  review=review.split()
  review=[stemmer.stem(word) for word in review if word not in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [ ]:
x=dt.loc[:,'Text']
y=dt["label"]
print(x.shape,",",y.shape)

(9865,) , (9865,)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer(max_features=3500)
# cv=CountVectorizer(max_features=3500)
x=cv.fit_transform(corpus).toarray()
print(x)
print(x.shape)
# enc=LabelEncoder()
# x=enc.fit_transform(x)
# print(x)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(9865, 3500)


In [ ]:
# sc=StandardScaler()
# x=sc.fit_transform(x.reshape(-1,1))
# print(x)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=43)
print(x_train.shape,",",x_test.shape)
print(y_train)

(6905, 3500) , (2960, 3500)
7661    Fake
242     Real
1965    Real
8099    Real
545     Fake
        ... 
8527    Fake
2065    Fake
8010    Real
2305    Real
3396    Fake
Name: label, Length: 6905, dtype: object


In [ ]:
# param_grid={
#     'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],  # Smoothing parameter
#     'fit_prior': [True, False]
# }
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
gridsearch=GridSearchCV(estimator=RandomForestClassifier(),param_grid=param_grid,cv=5,scoring='accuracy')
gridsearch.fit(x_train,y_train)
print("Best Hyperparameters: ",gridsearch.best_params_)
print("Best Accuracy Score: ",gridsearch.best_score_)

In [ ]:
classifier=RandomForestClassifier(criterion='gini',max_depth=100,n_estimators=100)
classifier.fit(x_train,y_train)
y_pred=classifier.predict(x_test)
print("y_test is \n",y_test)
print("y_pred is \n",y_pred)

In [ ]:
print("the accuracy is: \n",accuracy_score(y_test,y_pred)*100,"%")
print("classification report is: \n",classification_report(y_test,y_pred))
print("confusion matrix \n",confusion_matrix(y_test,y_pred))

In [ ]:
def prediction(n):
  n=[n]
  n=cv.transform(n).toarray()
  pred=classifier.predict(n)
  if pred==Real:
    print("the news is real")
  else:
    print("the news is fake")

n=input("enter the news: ")
result=prediction(n)
print(result)